In [1]:
import os, glob, math, random, json, warnings, itertools, time, gc
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneGroupOut
from tensorflow.keras.utils import to_categorical
import json
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR

warnings.filterwarnings('ignore')
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

# ------------------------------------------------------------------
RAW_DIR   = Path('data')
TRAIN_DIR = RAW_DIR / 'dataset_by_location'
TEST_CSV  = RAW_DIR / 'test.csv'                   # public test
LOCATION_IDS = ["right_arm", "left_arm", "right_leg", "left_leg"]
WORK_DIR = Path('work2')
WIN_SIZE  = 50
STRIDE    = 25
BATCH_SZ  = 256
EPOCHS    = 20
EMB_DIM   = 64
LR        = 1e-3
NUM_WORKERS = 4
DEVICE    = 'cpu'#cuda' if torch.cuda.is_available() else 'cpu'
label_map = {
    'null': 0,'jogging': 1,'jogging (rotating arms)': 2,'jogging (skipping)': 3,'jogging (sidesteps)': 4,'jogging (butt-kicks)': 5,
    'stretching (triceps)': 6,'stretching (lunging)': 7,'stretching (shoulders)': 8,'stretching (hamstrings)': 9,'stretching (lumbar rotation)': 10,
    'push-ups': 11,'push-ups (complex)': 12,'sit-ups': 13,'sit-ups (complex)': 14,'burpees': 15,'lunges': 16,'lunges (complex)': 17,'bench-dips': 18
}
loc = "right_arm"

2025-06-25 17:23:30.115234: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750865010.130127   75039 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750865010.134542   75039 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750865010.145391   75039 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750865010.145410   75039 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750865010.145412   75039 computation_placer.cc:177] computation placer alr

In [2]:
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import f1_score
from ast import literal_eval
from numpy import nan

class WearModelBuilder:
    def __init__(self, num_classes=19):
        self.num_classes = num_classes
        self.label_map = {
            'null': 0, 'jogging': 1, 'jogging (rotating arms)': 2,
            'jogging (skipping)': 3, 'jogging (butt-kicks)': 4,
            'stretching (triceps)': 5, 'stretching (lunging)': 6,
            'stretching (shoulders)': 7, 'stretching (hamstrings)': 8,
            'stretching (lumbar rotation)': 9, 'push-ups': 10,
            'push-ups (complex)': 11, 'sit-ups': 12,
            'sit-ups (complex)': 13, 'burpees': 14,
            'lunges': 15, 'lunges (complex)': 16,
            'bench-dips': 17
        }

    def load_data(self, file_path):
        with open(file_path) as f:
            df = pd.read_csv(f)
        df['x_axis'] = df['x_axis'].apply(lambda row: eval(row))
        df['y_axis'] = df['y_axis'].apply(lambda row: eval(row))
        df['z_axis'] = df['z_axis'].apply(lambda row: eval(row))
        df['sbj_id'] = df['sbj_id'].astype(int)
        df['label'] = df['label'].astype(int)
        return df

    def prepare_datasets(self, df, test_size=0.2, random_state=42):
        subjects = df['sbj_id'].unique()
        train_subjects, test_subjects = train_test_split(
            subjects, test_size=test_size, random_state=random_state
        )

        train_df = df[df['sbj_id'].isin(train_subjects)]
        test_df = df[df['sbj_id'].isin(test_subjects)]

        return train_df, test_df

    def build_conv_lstm_model(self, input_shape):
        """Build a ConvLSTM model based on challenge recommendations"""
        inputs = Input(shape=input_shape)

        # Conv block 1
        x = Conv1D(64, 5, activation='relu', padding='same')(inputs)
        x = BatchNormalization()(x)
        x = Dropout(0.3)(x)

        # Conv block 2
        x = Conv1D(64, 5, activation='relu', padding='same')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.3)(x)

        # LSTM layer
        x = LSTM(128, return_sequences=False)(x)
        x = Dropout(0.5)(x)

        # Output
        outputs = Dense(self.num_classes, activation='softmax')(x)

        model = Model(inputs=inputs, outputs=outputs)
        model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        return model

    def train_model(self, train_df, test_df, epochs=50, batch_size=32):
        """Train a model on the given data"""
        train_df["x_axis"]=train_df["x_axis"].apply(lambda row: np.array(row))
        train_df["y_axis"]=train_df["y_axis"].apply(lambda row: np.array(row))
        train_df["z_axis"]=train_df["z_axis"].apply(lambda row: np.array(row))
        test_df["x_axis"]=test_df["x_axis"].apply(lambda row: np.array(row))
        test_df["y_axis"]=test_df["y_axis"].apply(lambda row: np.array(row))
        test_df["z_axis"]=test_df["z_axis"].apply(lambda row: np.array(row))
        X_train = train_df[['x_axis', 'y_axis', 'z_axis']].values
        y_train = train_df['label'].values
        X_test = test_df[['x_axis', 'y_axis', 'z_axis']].values
        y_test = test_df['label'].values
        train_array = np.zeros((X_train.shape[0], 3, 50))
        for i in range(X_train.shape[0]):
            for j in range(X_train.shape[1]):
                train_array[i][j]=np.array(X_train[i][j])
        test_array = np.zeros((X_test.shape[0], 3, 50))
        for i in range(X_test.shape[0]):
            for j in range(X_test.shape[1]):
                test_array[i][j]=np.array(X_test[i][j])

        scaler = StandardScaler()
        X_train = scaler.fit_transform(np.array([x.flatten() for x in train_array])).reshape(train_array.shape)
        X_test = scaler.transform(np.array([x.flatten() for x in test_array])).reshape(test_array.shape)

        y_train = to_categorical(y_train, num_classes=self.num_classes)
        y_test = to_categorical(y_test, num_classes=self.num_classes)

        model = self.build_conv_lstm_model(input_shape=(3, 50))

        callbacks = [
            EarlyStopping(patience=10, restore_best_weights=True),
            ModelCheckpoint('best_model.h5', save_best_only=True)
        ]

        history = model.fit(
            X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=callbacks,
            verbose=1
        )

        y_pred = model.predict(X_test).argmax(axis=1)
        y_true = test_df['label'].values
        f1 = f1_score(y_true, y_pred, average='macro')
        print(f"Macro F1-Score: {f1:.4f}")
        return model, history, f1

In [84]:
wearModelBuilder =  WearModelBuilder()
df = wearModelBuilder.load_data(TRAIN_DIR/'left_arm_windows.csv')
result = wearModelBuilder.prepare_datasets(df)
model1 = wearModelBuilder.build_conv_lstm_model(result[0].shape)

In [45]:
wearModelBuilder =  WearModelBuilder()
df = wearModelBuilder.load_data(TRAIN_DIR/'right_arm_windows.csv')
result = wearModelBuilder.prepare_datasets(df)
model1 = wearModelBuilder.build_conv_lstm_model(result[0].shape)
model1

<Functional name=functional_7, built=True>

In [85]:
for entry in LOCATION_IDS[1:]:
    wear_model_builder = WearModelBuilder()
    df = wear_model_builder.load_data(TRAIN_DIR/f"{entry}_windows.csv")
    result = wearModelBuilder.prepare_datasets(df)
    wear_model_builder.build_conv_lstm_model(result[0].shape)
    model = wear_model_builder.train_model(result[0], result[1])
    model[0].save(WORK_DIR/f"model-{entry}.h5")

3403/3409 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6302 - loss: 1.0264

3409/3409 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.6302 - loss: 1.0264 - val_accuracy: 0.6472 - val_loss: 1.1035
Epoch 26/50
3409/3409 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.6288 - loss: 1.0298 - val_accuracy: 0.6285 - val_loss: 1.1511
Epoch 27/50
3409/3409 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.6283 - loss: 1.0250 - val_accuracy: 0.6511 - val_loss: 1.1155
Epoch 28/50
3409/3409 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.6368 - loss: 1.0146 - val_accuracy: 0.6549 - val_loss: 1.1058
Epoch 29/50
3409/3409 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.6324 - loss: 1.0200 - val_accuracy: 0.6421 - val_loss: 1.1339
Epoch 30/50
3409/3409 ━━━━━━━━━━━━━━━━━━━━ 17s 5ms/step - accuracy: 0.6332 - loss: 1.0143 - val_accuracy: 0.6509 - val_loss: 1.1393
Epoch 31/50
3409/3409 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - accuracy: 0.6359 - loss: 1.0069 - val_accuracy: 0.6431 - val_loss: 1.1434
Epoch 32/50
3409/3409 ━━━━━━━━━━━━━━━━━━━━ 22s 6ms/step - accuracy: 0.6351 - loss: 1.010

Macro F1-Score: 0.5403
Epoch 1/50
3399/3409 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4836 - loss: 1.5823

3409/3409 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.4837 - loss: 1.5817 - val_accuracy: 0.5363 - val_loss: 1.5846
Epoch 2/50
3409/3409 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.5649 - loss: 1.2518 - val_accuracy: 0.5644 - val_loss: 1.6420
Epoch 3/50
3409/3409 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.5851 - loss: 1.1823 - val_accuracy: 0.5545 - val_loss: 1.6064
Epoch 4/50
3409/3409 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - accuracy: 0.5902 - loss: 1.1551 - val_accuracy: 0.5618 - val_loss: 1.6509
Epoch 5/50
3409/3409 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.5993 - loss: 1.1282 - val_accuracy: 0.5810 - val_loss: 1.6265
Epoch 6/50
3409/3409 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - accuracy: 0.6039 - loss: 1.1144 - val_accuracy: 0.5663 - val_loss: 1.7094
Epoch 7/50
3409/3409 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.6043 - loss: 1.1064 - val_accuracy: 0.5748 - val_loss: 1.6976
Epoch 8/50
3409/3409 ━━━━━━━━━━━━━━━━━━━━ 15s 5ms/step - accuracy: 0.6102 - loss: 1.0930 - val

Macro F1-Score: 0.3804


In [46]:
model = wearModelBuilder.train_model(result[0],result[1])

3409/3409 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.6405 - loss: 1.0419 - val_accuracy: 0.6027 - val_loss: 1.2758
Epoch 37/50
3409/3409 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.6421 - loss: 1.0335 - val_accuracy: 0.5988 - val_loss: 1.2736
Epoch 38/50
3407/3409 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6415 - loss: 1.0319

3409/3409 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.6415 - loss: 1.0319 - val_accuracy: 0.6027 - val_loss: 1.2624
Epoch 39/50
3409/3409 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.6425 - loss: 1.0348 - val_accuracy: 0.5913 - val_loss: 1.2815
Epoch 40/50
3409/3409 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.6463 - loss: 1.0268 - val_accuracy: 0.6003 - val_loss: 1.2682
Epoch 41/50
3409/3409 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.6487 - loss: 1.0237 - val_accuracy: 0.5941 - val_loss: 1.2879
Epoch 42/50
3409/3409 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.6491 - loss: 1.0208 - val_accuracy: 0.5953 - val_loss: 1.2678
Epoch 43/50
3409/3409 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.6458 - loss: 1.0239 - val_accuracy: 0.5928 - val_loss: 1.2925
Epoch 44/50
3409/3409 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.6469 - loss: 1.0213 - val_accuracy: 0.6016 - val_loss: 1.2789
Epoch 45/50
3409/3409 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.6472 - loss: 1.020

In [50]:
test_data

,id,sbj_id,sensor_location,x_axis,y_axis,z_axis
0,0,22,right_arm,"[0.2428425714285714, 0.2134530714285714, 0.174...","[0.8463437142857143, 0.854221738095238, 0.8428...","[-0.4574545714285714, -0.4578691904761905, -0...."
1,1,22,right_arm,"[0.6752909910531751, 0.4247757439107542, 0.715...","[-0.9366750995461923, -0.6223881559309377, -0....","[-0.6175092749440796, -0.5824192453686647, -0...."
2,2,23,right_arm,"[0.04528874289978267, 0.02636863686942854, -0....","[0.5794056316368844, 0.5853422923083632, 0.646...","[0.5778143538332879, 0.5922718075175648, 0.873..."
3,3,23,right_arm,"[-0.4294609406487871, -0.6656510068112931, -0....","[-0.147632980836954, -0.014413570611691117, -0...","[-1.0071089537700262, -0.8174943548296542, -0...."
4,4,23,right_arm,"[2.476312835574577, 1.6733647285880453, 1.7557...","[-0.5983137218068052, -0.10164738944146791, -0...","[-2.0541462483911856, -1.6842387016850981, -1...."
...,...,...,...,...,...,...
48931,48931,23,left_leg,"[1.0630785714285715, 1.0709205714285714, 0.933...","[0.1038264761904761, 0.1515437142857142, 0.190...","[0.4514348571428571, 0.4639102857142857, 0.422..."
48932,48932,25,left_leg,"[0.834704380952381, 0.9560397142857142, 0.7693...","[0.4938431904761905, -0.2392702857142856, -0.9...","[0.0447854047619047, -0.0320387142857142, -0.2..."
48933,48933,24,left_leg,"[1.060091142857143, 1.0703054285714286, 1.0746...","[0.0607051428571428, 0.0170856904761904, 0.008...","[-0.1507257142857143, -0.1675727857142857, -0...."
48934,48934,23,left_leg,"[-0.9559132857142858, -0.0778062857142857, -0....","[-0.5109039047619047, 2.9042674285714285, 0.89...","[0.1249424761904761, 0.2449837142857142, 0.542..."


In [ ]:
import tensorflow as tf
test_df = pd.read_csv("./data/test.csv")
model_dict = {}
for entry in LOCATION_IDS:
    model_dict[entry] = tf.keras.models.load_model(WORK_DIR/f"model-{entry}.h5")
test_df["x_axis"]=test_df["x_axis"].apply(lambda row: np.array([float(x) for x in eval(row)]))
test_df["y_axis"]=test_df["y_axis"].apply(lambda row: np.array([float(x) for x in eval(row)]))
test_df["z_axis"]=test_df["z_axis"].apply(lambda row: np.array([float(x) for x in eval(row)]))
X_test = test_df[['x_axis', 'y_axis', 'z_axis']].values
test_array = np.zeros((X_test.shape[0], 3, 50))
for i in range(X_test.shape[0]):
    for j in range(X_test.shape[1]):
        test_array[i][j]=np.array(X_test[i][j])
scaler = StandardScaler()
X_test = scaler.fit_transform(np.array([x.flatten() for x in test_array])).reshape(test_array.shape)
results = []
for index in range(len(test_df['id'])):
    model = model_dict[test_df["sensor_location"][index]]
    result=model.predict(X_test[index].reshape((1, 3, 50)))
    results.append(np.argmax(result))

result_df = pd.DataFrame([list(test_df["id"].values), list(results)]).transpose().rename(columns={0: 'id', 1: 'label'})
result_df.to_csv("submission.csv", index=False)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

def load_models(location_ids, work_dir):
    return {
        loc: tf.keras.models.load_model(work_dir / f"model-{loc}.h5")
        for loc in location_ids
    }

def preprocess_df(df):
    for axis in ("x_axis", "y_axis", "z_axis"):
        df[axis] = df[axis].apply(lambda s: np.array([float(x) for x in eval(s)]))
    X = df[["x_axis", "y_axis", "z_axis"]].values
    N, D = X.shape[0], X.shape[1]
    arr = np.zeros((N, D, 50), dtype=float)
    for i in range(N):
        for j in range(D):
            arr[i, j] = X[i, j]
    return arr

def scale_array(arr):
    N, D, L = arr.shape
    flat = arr.reshape(N, D * L)
    scaled = StandardScaler().fit_transform(flat)
    return scaled.reshape(N, D, L)

def batch_predict(df, X_scaled, models):
    results = np.empty(len(df), dtype=int)

    for loc, model in models.items():
        mask = (df["sensor_location"] == loc).to_numpy()
        if not mask.any():
            continue

        X_loc = X_scaled[mask]               # Shape: (n_samples_loc, 3, 50)
        preds = model.predict(X_loc)         # Batch-Predict
        labels = np.argmax(preds, axis=1)    # Bestimme Klassen

        results[mask] = labels

    return results

test_df = pd.read_csv("./data/test.csv")

model_dict = load_models(LOCATION_IDS, WORK_DIR)

test_arr  = preprocess_df(test_df)
X_scaled  = scale_array(test_arr)

labels = batch_predict(test_df, X_scaled, model_dict)

result_df = pd.DataFrame({
    "id":    test_df["id"],
    "label": labels
})
result_df.to_csv("submission.csv", index=False)  # has 0.281


2025-06-25 17:25:02.724123: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


383/383 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
383/383 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
383/383 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
383/383 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [38]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv1D, LSTM, Dense, Dropout,
    BatchNormalization, Activation, Add,
    GlobalAveragePooling1D, concatenate, MaxPooling1D
)

class HARModels:
    @staticmethod
    def build_deepconvlstm(input_shape, num_classes):
        """Simplified DeepConvLSTM model"""
        inputs = Input(shape=input_shape)

        # Conv block 1
        x = Conv1D(64, 5, activation='relu', padding='same')(inputs)
        x = BatchNormalization()(x)
        x = Dropout(0.3)(x)

        # Conv block 2
        x = Conv1D(64, 5, activation='relu', padding='same')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.3)(x)

        # LSTM layer
        x = LSTM(128, return_sequences=False)(x)
        x = Dropout(0.5)(x)

        # Output
        outputs = Dense(num_classes, activation='softmax')(x)

        model = Model(inputs=inputs, outputs=outputs)
        return model

    @staticmethod
    def build_resnet(input_shape, num_classes):
        """1D ResNet model"""
        def residual_block(x, filters, kernel_size=3):
            # Shortcut
            shortcut = x

            # Main path
            x = Conv1D(filters, kernel_size, padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            x = Conv1D(filters, kernel_size, padding='same')(x)
            x = BatchNormalization()(x)

            # Add shortcut if dimensions match
            if shortcut.shape[-1] == filters:
                x = Add()([x, shortcut])

            x = Activation('relu')(x)
            return x

        inputs = Input(shape=input_shape)

        # Initial conv
        x = Conv1D(64, 7, padding='same')(inputs)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling1D(3)(x)

        # Residual blocks
        x = residual_block(x, 64)
        x = residual_block(x, 64)
        x = residual_block(x, 128)
        x = residual_block(x, 128)

        # Global pooling and output
        x = GlobalAveragePooling1D()(x)
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.5)(x)
        outputs = Dense(num_classes, activation='softmax')(x)

        model = Model(inputs=inputs, outputs=outputs)
        return model

    @staticmethod
    def build_inception_module(input_tensor, filters):
        """Single Inception module"""
        # Bottleneck
        x_bottleneck = Conv1D(filters, 1, padding='same', activation='relu')(input_tensor)

        # Branch 1: 1x1 conv
        branch1 = Conv1D(filters, 1, padding='same', activation='relu')(x_bottleneck)

        # Branch 2: 3x3 conv
        branch2 = Conv1D(filters, 3, padding='same', activation='relu')(x_bottleneck)

        # Branch 3: 5x5 conv
        branch3 = Conv1D(filters, 5, padding='same', activation='relu')(x_bottleneck)

        # Branch 4: Max pooling
        branch4 = MaxPooling1D(3, strides=1, padding='same')(input_tensor)
        branch4 = Conv1D(filters, 1, padding='same', activation='relu')(branch4)

        # Concatenate all branches
        return concatenate([branch1, branch2, branch3, branch4], axis=-1)

    @staticmethod
    def build_inceptiontime(input_shape, num_classes):
        """InceptionTime model"""
        inputs = Input(shape=input_shape)

        # Stack of inception modules
        x = HARModels.build_inception_module(inputs, 32)
        x = HARModels.build_inception_module(x, 32)
        x = HARModels.build_inception_module(x, 64)

        # Global pooling and output
        x = GlobalAveragePooling1D()(x)
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.5)(x)
        outputs = Dense(num_classes, activation='softmax')(x)

        model = Model(inputs=inputs, outputs=outputs)
        return model

### 1. Generate per‑location CSVs (adds `subject_id`)